#### 사용자 정의 모델 클래스
- 부모 클래스: nn.module
- 필수 오버라이딩
    - __init__(): 모델 구성 즉, 설계
    - forward(): 순방향 학습 진행 코드 구현
- 동적 모델
    - container 모듈 중 nn.ModulList() 사용해서 동적으로 Layer 추가
        - forward 기능 미 제공
        - layer 인스턴스 요소 사이에 연관성 없음
        - layer 인스턴스 요소는 인덱싱으로 접근

In [1]:
# 모듈로딩
import torch                                            # 텐서 관련 모듈
import torch.nn as nn                                   # 인공신경망 관련 모듈
import torch.nn.functional as F                         # 인공신경망 관련 함수들 모듈(손실,활성화 함수)
import torch.optim as optim                             # 최적화 관련 모듈(가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary                           # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *                   # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *               # 분류 성능 지표 관련 모듈



- [기본] 신경망클래스  <hr>
    - 입력층 -> 입력: 피쳐수 고정
    - 출력층 -> 출력: 타겟수 고정
    - 은닉층 -> 고정
    

In [2]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE='cuda' if torch.cuda.is_available() else 'cpu'

- 모델 설계 - 동적 모델 <hr>
    - 목표: 은닉층의 개수가 동적인 모델
    - 조건
        - 입력층과 출력층 개수 동적 => 입력층의 입력값, 출력층의 출력값
        - 은닉층의 개수 동적 => 퍼셉트론 개수 고정 => 은닉층의 개수, 은닉층 마다 퍼셉트론 수 
     


In [3]:
# 모델이름: DynamicModel
# 부모클래스: nn.Module
# 매개변수: in_in, out_out, h_ins=[], h_outs=[]

In [6]:
class DynamicModel(nn.Module):
    # 생성자 메서드
    def __init__(self, in_in, out_out, in_out, h_ins=[], h_outs=[]) -> None:
        super().__init__()

        self.in_layer=nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out)
        self.h_layers=nn.ModuleList()
        for idx in range(len(h_ins)):
            self.h_layers.append(nn.Linear(h_ins[idx], h_outs[idx]))
        self.out_layer=nn.Linear(h_outs[-1] if len(h_outs) else in_out, out_out)
    # 학습 진행 콜백 메서드
    def forward(self, x):
        y=self.in_layer(x)
        y=F.relu(y)
        
        for linear in self.h_layers:
            y= linear(y)
            y=F.relu(y)

        return self.out_layer(y)

In [7]:
# 모델 인스턴스 생성
h_ins, h_outs=[30,50,70], [50,70,30]
m1=DynamicModel(in_in=3, in_out=5, out_out=2, h_ins=h_ins, h_outs=h_outs)

In [8]:
# 모델 파라미터 확인
for name, param in m1.named_parameters():
    print(name, param)


in_layer.weight Parameter containing:
tensor([[-0.3492,  0.0725,  0.5754],
        [-0.3647,  0.3077, -0.3196],
        [-0.5428, -0.1227,  0.3327],
        [ 0.5360, -0.3586,  0.1253],
        [ 0.4982,  0.3826,  0.3598],
        [ 0.4103,  0.3652,  0.1491],
        [-0.3948, -0.4848, -0.2646],
        [-0.0672, -0.3539,  0.2112],
        [ 0.1787, -0.1307,  0.2219],
        [ 0.1866,  0.3525,  0.3888],
        [-0.1955,  0.5641, -0.0667],
        [-0.0198, -0.5449, -0.3716],
        [-0.3373, -0.2469,  0.4105],
        [-0.1887, -0.4314,  0.2221],
        [ 0.1848,  0.3739, -0.2988],
        [ 0.1252, -0.2102, -0.1297],
        [-0.4601, -0.2631, -0.1768],
        [ 0.2469,  0.1055,  0.1426],
        [ 0.5763,  0.5627,  0.3938],
        [ 0.0184, -0.3994,  0.4512],
        [-0.1444, -0.0467, -0.4974],
        [-0.1140, -0.3724,  0.5305],
        [-0.4991, -0.4500, -0.0196],
        [-0.3122,  0.2066, -0.2222],
        [-0.2712,  0.0327,  0.4179],
        [-0.4061,  0.2711,  0.3709],


- 학습진행


In [30]:
# 임시 데이터 생성
dataTS= torch.FloatTensor(([1,3,5], [2,4,6], [3,5,7], [4,6,8]))
targetTS= torch.FloatTensor([[7,9], [8,10], [9,11], [10,12]])

In [1]:
#  모델 학습 
# pre_y=m1(dataTS)
# print(pre_y)

